In [ ]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [ ]:
import logging
import delta_sharing
import pandas as pd

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [6]:
accounts = return_account_ids()

In [4]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.monitoring.datashifts" #features_view_30_outlook"
df = delta_sharing.load_as_pandas(table_path)#, limit=200000)
path_to_save = "/Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/SprintStories/PRD-2581-Pricing/"

In [5]:
df

In [11]:
def add_account(account_id, accounts):
    item = [acc for acc in accounts if acc["id"] == account_id]
    if len(item) == 1:
        return item[0]["name"]
    else:
        return None
df["account"] = df["account_id"].apply(lambda x: add_account(x, accounts))
df

In [12]:
df["month"] = pd.to_datetime(df["created"]).dt.strftime("%Y-%m")
df

In [13]:
df = df.drop_duplicates(subset=["account_id","max_date","count_anonymouIds_max_date","count_sessions_max_date"])
df = df.reset_index(drop=True)
df

In [16]:
df["unique_sessions"] = df.groupby(by=["account_id","month"])["count_sessions_max_date"].transform("sum")
df["unique_anonymousIds"] = df.groupby(by=["account_id","month"])["count_anonymouIds_max_date"].transform("sum")
df

In [20]:
df = df.drop_duplicates(subset=["account","month","unique_sessions","unique_anonymousIds"])
df = df[["account","month","unique_sessions","unique_anonymousIds"]]
df = df.sort_values(by=["account","month"])
df

In [21]:
df.to_csv(f"{path_to_save}sessions_anonymousIds_per_month_datashift.csv")